# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [39]:
# import libraries

# 1) read SQL
import pandas as pd
from sqlalchemy import create_engine

# 2) tokenization function 
import nltk
import re
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

#3) machine learning pipeline
from sklearn.pipeline import Pipeline  # For creating the pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer  # For text processing
from sklearn.multioutput import MultiOutputClassifier  # For multi-output classification
from sklearn.ensemble import RandomForestClassifier  # For the Random Forest classifier

#4) training of pipeline
from sklearn.model_selection import train_test_split

#5 test training model
from sklearn.metrics import classification_report
import numpy as np

#8 Improve model with Grid Search
from sklearn.model_selection import GridSearchCV        #for using GridSearchCV


#9 Export your model as a pickle file
import pickle     # for ML-model export s a pickle file


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
# 1. load data from database.

# IMPORTANT: Notebook from 5.5 ETL has to be run bevore!
# load data from database

engine = create_engine('sqlite:///DisasterResponseProject.db')
df = pd.read_sql('SELECT * FROM DisasterResponses', engine)
X = df['message']
y = df.iloc[:,4:]


In [41]:
df.head() #check df structure.


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Write a tokenization function to process your text data

In [42]:
# Check text in column 'message'
list(df['message'][:20])

['Weather update - a cold front from Cuba that could pass over Haiti',
 'Is the Hurricane over or is it not over',
 'Looking for someone but no name',
 'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
 'says: west side of Haiti, rest of the country today and tonight',
 'Information about the National Palace-',
 'Storm at sacred heart of jesus',
 'Please, we need tents and water. We are in Silo, Thank you!',
 'I would like to receive the messages, thank you',
 'I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )',
 "There's nothing to eat and water, we starving and thirsty.",
 'I am in Petionville. I need more information regarding 4636',
 'I am in Thomassin number 32, in the area named Pyron. I would like to have some water. Thank God we are fine, but we desperately need water. Thanks',
 "Let's do it together, need food in Delma 75, in didine area",
 'More informati

In [43]:
# 2. Write a tokenization function to process your text data


# Check text in column 'message'
list(df['message'][:20])

def tokenize(text):
    # Define a regex pattern to detect URLs
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Replace URLs with a placeholder
    text = re.sub(url_regex, "urlplaceholder", text)
    
    # Normalize and tokenize text
    tokens = word_tokenize(text.lower())
    
    # Remove stopwords
    tokens = [w for w in tokens if w not in stopwords.words("english") and w.isalpha()]
    
    # Initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Iterate through each token
    clean_tokens = []
    for tok in tokens:
        # Lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


In [44]:
# Check text tokens in column 'message'
for message in X[:20]:
    tokens = tokenize(message)
    print(tokens,'\n')


['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

['hurricane'] 

['looking', 'someone', 'name'] 

['un', 'report', 'leogane', 'destroyed', 'hospital', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 

['information', 'national'] 

['storm', 'sacred', 'heart', 'jesus'] 

['please', 'need', 'tent', 'water', 'silo', 'thank'] 

['would', 'like', 'receive', 'message', 'thank'] 

['health', 'issue', 'worker', 'santo', 'area'] 

['nothing', 'eat', 'water', 'starving', 'thirsty'] 

['petionville', 'need', 'information', 'regarding'] 

['thomassin', 'number', 'area', 'named', 'pyron', 'would', 'like', 'water', 'thank', 'god', 'fine', 'desperately', 'need', 'water', 'thanks'] 

['let', 'together', 'need', 'food', 'delma', 'didine', 'area'] 

['information', 'number', 'order', 'participate', 'see', 'use'] 

['comitee', 'delmas', 'rue', 'street', 'janvier', 'impasse', 'charite', 'people', 't

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [45]:
# Replace all NaN values with 0

df.fillna(0, inplace=True)

# split the dataset

X = df.message
y = df.iloc[:,4:]
category_names = y.columns


In [46]:
#3. Build a machine learning pipeline

machine_learning_pipeline = Pipeline([
    ('cvect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [47]:
# 4. Train pipeline

X_train, X_test, y_train, y_test = train_test_split(X, y)
machine_learning_pipeline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [48]:
y_pred = machine_learning_pipeline.predict(X_test)

# Test shape of y_test and y_pred
print("Shape of y_test:", y_test.values.shape)
print("Shape of y_pred:", y_pred.shape)

# Test unique values in y_test and y_pred
print("Unique values in y_test:", np.unique(y_test))
print("Unique values in y_pred:", np.unique(y_pred))


Shape of y_test: (6554, 36)
Shape of y_pred: (6554, 36)
Unique values in y_test: [ 0.  1.  2.]
Unique values in y_pred: [ 0.  1.  2.]


In [49]:
# Generate predictions y_pred and print out classification_report for all 36 categories 
y_pred = machine_learning_pipeline.predict(X_test)

# Iterate through each column and print the classification report
for i in range(y_test.shape[1]):  # Use the number of columns in y_test
    print(i+1,")","#########################", y_test.columns[i], "#########################")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))
  

1 ) ######################### related #########################
             precision    recall  f1-score   support

        0.0       0.61      0.46      0.53      1546
        1.0       0.85      0.90      0.87      4954
        2.0       0.28      0.48      0.35        54

avg / total       0.79      0.80      0.79      6554

2 ) ######################### request #########################
             precision    recall  f1-score   support

        0.0       0.89      0.98      0.93      5419
        1.0       0.80      0.44      0.57      1135

avg / total       0.88      0.88      0.87      6554

3 ) ######################### offer #########################
             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      6520
        1.0       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

4 ) ######################### aid_related #########################
             precision    recall  f1-score  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [53]:
# Specify parameters for grid search
parameters = {
    'clf__estimator__n_estimators': [100],           # used for shorter run time  
#   'clf__estimator__n_estimators': [100, 200],      # better but runs longer     
    'clf__estimator__min_samples_split': [2, 3],
}

# Create grid search object
cv = GridSearchCV(machine_learning_pipeline, param_grid=parameters, return_train_score=True, verbose=2)

cv.fit(X_train, y_train)



Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 4.7min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.9min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 4.7min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 4.8min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100, total= 4.3min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100, total= 4.2min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100, total= 4.2min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 34.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [100], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=2)

In [54]:
#building new model

machine_learning_pipeline_optimized = cv.best_estimator_
print (cv.best_estimator_)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [55]:
# Generate predictions y_pred and print out classification_report
y_pred = machine_learning_pipeline_optimized.predict(X_test)  # test of the optimized machine learning model

# Iterate through each column and print the classification report
for i in range(y_test.shape[1]):  # Use the number of columns in y_test
    print(i,")","#########################", y_test.columns[i], "#########################")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))


0 ) ######################### related #########################
             precision    recall  f1-score   support

        0.0       0.70      0.42      0.52      1546
        1.0       0.84      0.94      0.89      4954
        2.0       0.31      0.52      0.39        54

avg / total       0.80      0.81      0.80      6554

1 ) ######################### request #########################
             precision    recall  f1-score   support

        0.0       0.90      0.98      0.94      5419
        1.0       0.82      0.50      0.62      1135

avg / total       0.89      0.89      0.88      6554

2 ) ######################### offer #########################
             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      6520
        1.0       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

3 ) ######################### aid_related #########################
             precision    recall  f1-score  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

        0.0       0.89      0.95      0.92      4764
        1.0       0.85      0.69      0.77      1790

avg / total       0.88      0.88      0.88      6554

29 ) ######################### floods #########################
             precision    recall  f1-score   support

        0.0       0.95      1.00      0.97      6028
        1.0       0.90      0.46      0.61       526

avg / total       0.95      0.95      0.95      6554

30 ) ######################### storm #########################
             precision    recall  f1-score   support

        0.0       0.95      0.99      0.97      5980
        1.0       0.78      0.50      0.61       574

avg / total       0.94      0.94      0.94      6554

31 ) ######################### fire #########################
             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99      6485
        1.0       0.50      0.01      0.03        69

avg /

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [56]:
pickle.dump(machine_learning_pipeline_optimized, open('model.pkl', 'wb'))


### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [57]:
# TESTING the pickl function

# Load the model
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Check if the model has been loaded
print(loaded_model)

# Test the loaded model: make predictions with the loaded model. Use the same input data used for training the model.

# Example input data
X = ["There's a huge fire in the forest. Nothing to eat and water; we are starving and thirsty."]
prediction = loaded_model.predict(X)
print(prediction)


Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])
[[ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.]]
